In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import sys

In [ ]:
import os

In [ ]:
import re

In [ ]:
#basedir = "/home/marco/git/webvalley/datapreproc"
basedir = "C:/Users/julix/webvalley_git/full_project/data/PLIC-Milano"
#basedir = "/home/mattia/Scrivania/plic_clinical_data/plic_clinical_data/PLIC-Milano"

# Import the smallest dataset

The plic-milano-foglio-piccolo.xlsx is PLIC_1445_V1_V2_V3_V4_linear variables _ updated 15_01_19.xlsx
and the plic-milano-foglio-grande.xlsx is PLIC_dataset parametri clinici_V1_V2_V3_V4_1445.xlsx

In [ ]:
smalldf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-piccolo.xlsx"), index_col=0).fillna(-1)

In [ ]:
smalldf.head()

In [ ]:
smalldf.info()

# What is male and what is female?
### Should at least be written in the docs of the whole software otherwise it will be impossible to import new data

In [ ]:
smalldf.sesso.unique()

# Import the biggest dataset

In [ ]:
bigdf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-grande.xlsx"), index_col=0).fillna(-1)

In [ ]:
bigdf.info()

In [ ]:
bigdf.head()

# At least one column is not in the biggest dataset

In [ ]:
smalldfcols = set(smalldf.columns.values)

In [ ]:
bigdfcols = set(bigdf.columns.values)

In [ ]:
f1_cols_in_f2 = smalldfcols.intersection(bigdfcols)

In [ ]:
f1_cols_non_in_f2 = smalldfcols.difference(bigdfcols)

In [ ]:
f1_cols_in_f2

In [ ]:
f1_cols_non_in_f2

In [ ]:
len(f1_cols_in_f2)

In [ ]:
len(f1_cols_non_in_f2)

Checking if column names differ because of case sensitivity

In [ ]:
bigdfcols_lower = set([k.lower() for k in bigdfcols])

In [ ]:
smalldfcols_lower = set([k.lower() for k in smalldfcols])

In [ ]:
len(smalldfcols_lower.intersection(bigdfcols_lower))

which is not the case

# Check how many patients in the smaller dataset are also in the biggest one

In [ ]:
paz_in_smalldf = set(smalldf.index.values)

In [ ]:
paz_in_bigdf = set(bigdf.index.values)

In [ ]:
paz_in_smalldf.difference(paz_in_bigdf)

In [ ]:
paz_in_bigdf.difference(paz_in_smalldf)

*All the patients in the small dataset are also in the biggest one*

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()

# Fix known issues with the dataset

The two datasets uses different ways to store the data (M/F -> 0/1; Yes/No -> 1/0)

In [ ]:
smalldf["sesso"] = smalldf["sesso"].map({1: "F", 0: "M"})

*We dropped all sex errors so the method is appropriate; now replace even Si and No*

In [ ]:
# Check all the columns that contains at least one Si or No

bool_cols = list()

for col in f1_cols_in_f2:
    k = getattr(bigdf, col).values
    if np.isin("Sì", k) or np.isin("No", k):
        bool_cols.append(col)

In [ ]:
bool_cols

In [ ]:
# Replace em all

for col in bool_cols:
    smalldf[col] = smalldf[col].map({0: "No", 1: "Sì"})

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()


## Now check if dataset 1 and dataset 2 shared patients contains same data

In [ ]:
# Pick every patient in the big dataset and check if the values are the same
# that are stored in the smaller one

for paz_to_be_checked in paz_in_smalldf:
    paz_from_big = bigdf.loc[paz_to_be_checked]
    paz_from_small = smalldf.loc[paz_to_be_checked]
    if not np.array_equal(paz_from_big[f1_cols_in_f2].values, paz_from_small[f1_cols_in_f2].values):
        print("Paz %s has differences" % paz_to_be_checked)

print("check complete")

# Adding non-duplicate data from smaller dataframe to bigger dataframe

## Preparing smaller dataframe to join with bigger dataframe

In [ ]:
smalldf.info()

In [ ]:
smalldf.drop(f1_cols_in_f2, axis=1, inplace=True)

In [ ]:
smalldf.info()

In [ ]:
bigdf.info()

In [ ]:
joined_df = bigdf.join(smalldf, how='outer')

In [ ]:
joined_df.info()
joined_df['menarca_1'].unique()

# Get rid of empty columns

In [ ]:
cols_to_drop = list()
for i in joined_df.columns.values:
    o = joined_df[i].unique()
    if len(o) == 1 and o[0] == -1:
        cols_to_drop.append(i)

cols_to_drop

In [ ]:
joined_df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
joined_df.info()

# Remove columns ```_<int:pk>``` and divide patients visits on different rows

In [ ]:
[x for x in joined_df.columns]

In [ ]:
suffixes = ["_%s" % a for a in range(0,10)] + \
           ["_%s_a" % a for a in range(0,10)] + \
           ["_%s_recod" % a for a in range(0,10)]

In [ ]:
suffixes

In [ ]:
mv_cols = [x for x in joined_df.columns if x.endswith(tuple(suffixes))]

In [ ]:
mv_cols

In [ ]:
len(mv_cols)

In [ ]:
perdurant_cols = [x for x in joined_df.columns if x not in mv_cols]

In [ ]:
new_cols = set()

for col in mv_cols:
    for s in suffixes:
        col = col.replace(s, "")
    col.replace("__", "_")
    if col.endswith("_"):
        col = col[:-1]
    new_cols.add(col)

for col in perdurant_cols:
    new_cols.add(col)

new_cols.add("cod_pz")
new_cols

In [ ]:
len(new_cols)

# Create a new dataframe with the defined columns

In [ ]:
fields_per_visit = set()

for col in mv_cols:
    for s in suffixes:
        col = col.replace(s, re.sub("\d", "%s", s))
    fields_per_visit.add(col)

fields_per_visit

In [ ]:
# Test
for col in [a % b for a in fields_per_visit for b in range(1, 5)]:
    if col not in joined_df.columns.values:
        print("Cannot find", col)


In [ ]:
# Check we do have the same number of cols as before
len([k for k in [a % b for a in fields_per_visit for b in range(1, 5)] if k in joined_df.columns])

## Fill the new dataframe with the data from the other one

In [ ]:
# Build a dictionary to convert multicols names into singlecol
single = dict()

for col in mv_cols:
    _col = col
    for s in suffixes:
        col = col.replace(s, "")
    col.replace("__", "_")
    if col.endswith("_"):
        col = col[:-1]
    single[_col] = col

In [ ]:
all([k in new_cols for k in single.values()])

In [ ]:
new_data = []

for paz in joined_df.index.values:
        obj = joined_df.loc[paz]
        for visit in range(1, 5):
            this = {"cod_pz": paz}
            for old_col in fields_per_visit:
                try:
                    this[single[old_col % visit]] = obj[old_col % visit]
                except KeyError:
                    pass
            for pd_col in perdurant_cols:
                this[pd_col] = obj[pd_col]
            new_data.append(this)

In [ ]:
len(new_data)

In [ ]:
pippo = pd.DataFrame(new_data, columns=new_cols).fillna(-1)

In [ ]:
pippo.info()

In [ ]:
joined_df.info()

In [ ]:
pippo.head()

In [ ]:
k = set()
mask_fields = set()
for col in pippo.columns.values:
    if len(pippo[col].unique())<10:
        print(col, pippo[col].unique())
        mask_fields.add(col)

In [ ]:
rp = {
    "NR": -1,
    "campo vuoto": -1,
    "mancante": -1,
    "No": 0,
    "Sì": 1,
    "Si": 1,
    "no": 0,
    "sì": 1,
    "si": 1,
    "presenti": 1,
    "assenti": 0,
    "assente": 0,
    "completo": 1,
    "incompleto": 2,
    "sì - saltuario": 1,
    "sì - regolare": 2,
    "non palpabile": 0,
    "palpabile": 1,
    "normale": 0,
    "patologico": 1,
    "nessuna": 0,
    "leggera": 1,
    "media": 2,
    "pesante": 3,
    "ex": 2,
    "F": 1,
    "M": 0,
}

In [ ]:
for col in mask_fields:
    if pippo[col].dtype == np.object_:
        pippo[col] = pippo[col].replace(rp)

In [ ]:
pippo.head()

# Translate column names

In [ ]:
ita_clist = list(pippo.columns.values)

In [ ]:
ita_clist = sorted(ita_clist, key=lambda s: s.lower())

In [ ]:
ita_clist

In [ ]:
print("\n".join([x.replace("_a", "").replace("_", " ") for x in ita_clist]))

### Translation from DeepL


```
ABI dx calc
ABI mean calc
ABI sx calc
abortion n
abdomen
abdomen type
AGRATIO
alcohols
HIGH
height
other pathologies
Android perc
angina
angina date
angina profile
remote angina a
antiaggregants to
antidiabetics to
antihypertensive to
aorta calcifications
apoA1
apoB
respiratory system
right corneal arch
left corneal arch
arrhythmia
arrhythmia date
major arrhythmias
major arrhythmias date
aortic arteriopathy
aortic arteriopathy date
remote aortic arteriopathy a
peripheral arteriopathy
peripheral arteriopathy limbs
peripheral arteriopathy given limbs
peripheral arteriopathy given
remote peripheral arteriopathy a
peripheral arteriopathy TSA
peripheral arteriopathy TSA given
extremities
AST
ATT ATC1
ATT ATC2
ATT ATC3
ATT start date1
ATT start date2
ATT start date3
ATT indication1
ATT PA1
ATT PA2
ATT PA3
physical activity
physical activity intensity
physical activity hours
b Beer
b spirits
b wine
basophils
basophils percent
BMC g
BMD g
BMI calc
body scan data
COPD
COPD date
bypass
bypass date
Profile bypass
remote bypass to
CHD
CHD profile
Remote CHD to
clearance CG calc
cod pcs
cholecystectomy
cholecystectomy date
conc hb cell medium
CPK
creatinine
TORQUE AND ANGLE WRENCH
cyanosis skin
cute oedemas
skin hematomas
skin infections
pale skin
tofi skin
cute xanthelasms
Cute Xanthomas
date endothelium
date of withdrawal
date TIA
date of visit
diabetes1 self
diabetes1 self data
diabetes2 self
diabetes2 self data
diagnosis of new revaluations
endothelial dysfunction
self dyslipidaemia
self data dyslipidemia
self type dyslipidemia
DM ATC1
DM ATC2
DM ATC3
DM ATC4
DM start date1
DM start date2
DM start date3
DM start date4
DM PA1
DM PA2
DM PA3
DM PA4
Dys ATC1
Dys ATC2
Dys start date1
Dys start date2
Dys PA1
Dys PA2
EA
EA Low
ECOSTEATOSIS ECOCARDIO
hematocrit
haemoglobin
hemoglobin cell medium
cerebral hemorrhage
cerebral hemorrhage given
eosinophils
eosinophils percent
hepatopathies date
hepatopathies NS
hepatopathies type
erythrocytes
ages
NS Drugs
FE
enlarged liver
flanks
fibers a
atrial fibrillation
given atrial fibrillation
FMD
heart rate
smoking
smoking recod
gGT
ginoid perc
glucose
epicardial fat
fat g
pregnancies n
completed pregnancies
HDL
HT ATC1
HT ATC2
HT ATC3
HT ATC4
HT ATC5
HT start date1
HT start date2
HT start date3
HT start date4
HT indication1
HT PA1
HT PA2
HT PA3
HT PA4
HT PA5
strokes
stroke date
Stroke Profile
Remote stroke at
IMA
IMA date
IMA profile
Remote IMA at
IMT DC max
IMT CC max right
IMT CC max left
IMT DC medium
IMT DC medium right
IMT CC medium left
diagnostic investigations1 date
diagnostic investigations1 outcome
diagnostic investigations1 type examination
diagnostic investigations2 date
diagnostic investigations2 outcome
diagnostic investigations2 type examination
diagnostic investigations3 date
diagnostic investigations3 outcome
diagnostic investigations3 type examination
induced
hypertension self
hypertension self data
silent ischaemia
ischemia silente date
ischaemia silent profile
remote silent ischemia a
learning
IVS
IVS date
LDL calc
leukocytes
lymphocytes
lymphocytes percent
Lunar NS
lean g
menarche
menopause
menopause age
monocytes
monocytes percent
MVS
emergence
nephropathy date
nephropathies NS
nephropathies profile
nephropathies type
neoplasm1 date
neoplasm1 NS
neoplasm1 operated on
neoplasm1 therapy
neoplasm1 type
neoplasm2 given
neoplasm2 NS
neoplasm2 operated
neoplasia2 therapy
neoplasm2 type
given neoplasm
NS neoplasm
type neoplasm
neutrophils
neutrophils percent
well-known
known endothelium
self obesity
obesity self data
OT ATC1
USED OT ATC10
OT ATC11
OT ATC12
OT ATC2
OT ATC3
OT ATC4
OT ATC5
OT ATC6
OT ATC7
OT ATC8
OT ATC9
PA ankle right
PA left ankle
pad
pancreas
pancreatopathy given
pancreatopathy NS
pancreatopathy type
pas
PCR
burden
plates
PLACA
right plate NS
left plate NS
peripheral arterial wrists
SECONDARY PREVENTION
PTCA
PTCA date
PTCA profile
Remote PTCA to
RDW CV
RDW SD
imbalance
decompensation date
remote decompensation at
gender
SIV
NS blows
blow type
statins a
steatosis
steatosis degree
steatosis NS
fabric g
fabric perc
TG
TIA
TIA profile
Remote TIA to
thyroid appearance
thyroid function
thyroid nodules
thyroid pathologies date
thyroid pathologies text
venous thrombosis
venous thrombosis given
U TSA date
uricemia
urine date
visits carried out
standardised visits to
lifetime
waist hips calc
vol cell medium
xanthomatosis
```

In [ ]:
en_clist = """ABI dx calc
ABI mean calc
ABI sx calc
abortion n
abdomen
abdomen type
AGRATIO
alcohols
HIGH
height
other pathologies
Android perc
angina
angina date
angina profile
remote angina a
antiaggregants to
antidiabetics to
antihypertensive to
aorta calcifications
apoA1
apoB
respiratory system
right corneal arch
left corneal arch
arrhythmia
arrhythmia date
major arrhythmias
major arrhythmias date
aortic arteriopathy
aortic arteriopathy date
remote aortic arteriopathy a
peripheral arteriopathy
peripheral arteriopathy limbs
peripheral arteriopathy given limbs
peripheral arteriopathy given
remote peripheral arteriopathy a
peripheral arteriopathy TSA
peripheral arteriopathy TSA given
extremities
AST
ATT ATC1
ATT ATC2
ATT ATC3
ATT start date1
ATT start date2
ATT start date3
ATT indication1
ATT PA1
ATT PA2
ATT PA3
physical activity
physical activity intensity
physical activity hours
b Beer
b spirits
b wine
basophils
basophils percent
BMC g
BMD g
BMI calc
body scan data
COPD
COPD date
bypass
bypass date
Profile bypass
remote bypass to
CHD
CHD profile
Remote CHD to
clearance CG calc
cod pcs
cholecystectomy
cholecystectomy date
conc hb cell medium
CPK
creatinine
total cholesterol
cyanosis skin
cute oedemas
skin hematomas
skin infections
pale skin
tofi skin
cute xanthelasms
Cute Xanthomas
date endothelium
date of withdrawal
date TIA
date of visit
diabetes1 self
diabetes1 self data
diabetes2 self
diabetes2 self data
diagnosis of new revaluations
endothelial dysfunction
self dyslipidaemia
self data dyslipidemia
self type dyslipidemia
DM ATC1
DM ATC2
DM ATC3
DM ATC4
DM start date1
DM start date2
DM start date3
DM start date4
DM PA1
DM PA2
DM PA3
DM PA4
Dys ATC1
Dys ATC2
Dys start date1
Dys start date2
Dys PA1
Dys PA2
EA
EA Low
ECOSTEATOSIS ECOCARDIO
hematocrit
haemoglobin
hemoglobin cell medium
cerebral hemorrhage
cerebral hemorrhage given
eosinophils
eosinophils percent
hepatopathies date
hepatopathies NS
hepatopathies type
erythrocytes
age
NS Drugs
FE
enlarged liver
flanks
fibers a
atrial fibrillation
given atrial fibrillation
FMD
heart rate
smoking
smoking recod
gGT
ginoid perc
glucose
epicardial fat
fat g
pregnancies n
completed pregnancies
HDL
HT ATC1
HT ATC2
HT ATC3
HT ATC4
HT ATC5
HT start date1
HT start date2
HT start date3
HT start date4
HT indication1
HT PA1
HT PA2
HT PA3
HT PA4
HT PA5
strokes
stroke date
Stroke Profile
Remote stroke at
IMA
IMA date
IMA profile
Remote IMA at
IMT DC max
IMT CC max right
IMT CC max left
IMT DC medium
IMT DC medium right
IMT CC medium left
diagnostic investigations1 date
diagnostic investigations1 outcome
diagnostic investigations1 type examination
diagnostic investigations2 date
diagnostic investigations2 outcome
diagnostic investigations2 type examination
diagnostic investigations3 date
diagnostic investigations3 outcome
diagnostic investigations3 type examination
induced
hypertension self
hypertension self data
silent ischaemia
ischemia silente date
ischaemia silent profile
remote silent ischemia a
learning
IVS
IVS date
LDL calc
leukocytes
lymphocytes
lymphocytes percent
Lunar NS
lean g
menarche
menopause
menopause age
monocytes
monocytes percent
MVS
birth
nephropathy date
nephropathies NS
nephropathies profile
nephropathies type
neoplasm1 date
neoplasm1 NS
neoplasm1 operated on
neoplasm1 therapy
neoplasm1 type
neoplasm2 given
neoplasm2 NS
neoplasm2 operated
neoplasia2 therapy
neoplasm2 type
given neoplasm
NS neoplasm
type neoplasm
neutrophils
neutrophils percent
well-known
known endothelium
self obesity
obesity self data
OT ATC1
USED OT ATC10
OT ATC11
OT ATC12
OT ATC2
OT ATC3
OT ATC4
OT ATC5
OT ATC6
OT ATC7
OT ATC8
OT ATC9
PA ankle right
PA left ankle
pad
pancreas
pancreatopathy given
pancreatopathy NS
pancreatopathy type
pas
PCR
burden
plates
plaque
right plate NS
left plate NS
peripheral arterial wrists
SECONDARY PREVENTION
PTCA
PTCA date
PTCA profile
Remote PTCA to
RDW CV
RDW SD
imbalance
decompensation date
remote decompensation at
gender
SIV
NS blows
blow type
statins a
steatosis
steatosis degree
steatosis NS
fabric g
fabric perc
TG
TIA
TIA profile
Remote TIA to
thyroid appearance
thyroid function
thyroid nodules
thyroid pathologies date
thyroid pathologies text
venous thrombosis
venous thrombosis given
U TSA date
uricemia
urine date
visits carried out
standardised visits to
lifetime
waist hips calc
vol cell medium
xanthomatosis""".split("\n")

In [ ]:
assert len([a for a in pippo]) == len(en_clist)

In [ ]:
trans_dict = {"given":"date", 
              "data" : "date", 
              "IMT DC":"IMT CC", 
              "learning":"education", 
              "cod_pcs":"subject_id", 
              "cod_pz":"subject_id", 
              "fabric":"tissue", 
              "blow":"bruit", 
              "blows":"bruit", 
              "flanks":"hips", 
              "well-known":"notes", 
              "withdrawal":"sampling", 
              "burden":"weight", 
              "pas":"SBP", 
              "pas":"DPB", 
              "MVS":"LBM",
              "medium":"average", 
              "known endothelium":"endothelium notes",
              "AGRATIO":"AG-RATIO", 
              "Agratio":"AG-ratio"
              }

In [ ]:
sl = ["self", "date"]

In [ ]:
en_clist

In [ ]:
for i in en_clist:
    for w in trans_dict:
        if w in i:
            i.replace(w, trans_dict[w])
        for s in sl:
            if s in i:
                i.replace(s, "")
                i = i + " " + s
                i.replace("  ", " ")
        
        

In [ ]:
en_clist

In [ ]:
pippo = pippo[ita_clist]

In [ ]:
pippo.head()

In [ ]:
pippo.columns = [en_clist]

In [ ]:
pippo.head()

In [ ]:
pippo.to_csv(os.path.join(basedir, "PLIC-milano-processed-and-translated.csv"), sep=";")
#pippo.to_csv("/out_Milano.csv")

In [ ]:
print("The end.")